# <a style='color:cyan'>Генерация протокола встречи</a>

<img src='data/img/logo.gif' width=400>

___
## <a>Содержание</a>
- [Импорт библиотек](#0)
    - GigaChat credentials
    - langchain
- [Предобработка текста](#1)
    - Импорт файла
    - Разделение текста
    - Векторизация текста
    - База данных векторов
- [Модель](#2)
    - Инициализация модели
    - Система вопрос-ответ
    - Промт
    - Экспорт в файл

___
## <center> <a id=0>Импорт библиотек</a>


In [5]:
from config import CREDENTIALS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import RetrievalQA

___
## <center> <a id=1>Предобработка текста</a>

#### $\boxed{\text{Импорт файла}}$

In [1]:
with open('data/input/text.md', encoding='utf-8') as f:
    text = f.read()

#### $\boxed{\text{Разделение текста}}$

In [ ]:
text_splitter = CharacterTextSplitter(
        separator=r"\n",
        is_separator_regex=True,
        chunk_size=500
    )

splitted_text = text_splitter.split_text(text)

#### $\boxed{\text{Векторизация текста}}$

`sbert_large_nlu_ru` -  SBERT-модель, для создания эмбеддингов, обученная на русскоязычных текстах. Разработана SberDevices.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="ai-forever/sbert_large_nlu_ru")

#### $\boxed{\text{База данных векторов}}$

In [ ]:
db_td = FAISS.from_texts(texts=splitted_text, embedding=embeddings)

___
## <center> <a id=2>Модель</a>

#### $\boxed{\text{Инициализация модели}}$

GigaChat - сервис на основе большой языковой модели, поддерживающий русский язык. Разработан SberDevices и Sber AI .


In [5]:
llm = GigaChat(credentials=CREDENTIALS, verify_ssl_certs=False)

#### $\boxed{\text{Система вопрос-ответ}}$

In [6]:
answers = RetrievalQA.from_chain_type(llm, retriever=db_td.as_retriever())

#### $\boxed{\text{Промт}}$

In [60]:
chapters = 'Участники, Темы, Ключевые решения, Важниые даты'

input_text =  f"Тебе дан текст описания встречи с временными тегами. Строго на основании данного описания встречи, сформируй протокол встречи, состоящий из разделов: {chapters}. В заглавии напиши: 'Протокол встречи'.\
    В разделе:\
        Участники, выдели участников встречи, для дальнейшего анализа встречи,\
        Темы, выдели главные уникальные темы, под кажой темой перечисли затронутые важные вопросы\
        Ключевые решения, перечисли все ключевые решения, в противном случае напиши: 'Ключевые решения не принимались.',\
        Важниые даты, перечисли даты всех запланированных мероприятий, встреч, дедлайнов, в противном случае напиши: 'Мероприятия не планировались'.\
        В разделе Темы для каждой темы перечисли затронутые вопросы\
            "
        
result = answers({"query": input_text})['result']
print(result)

Протокол встречи

Участники:
- Спикер А
- Спикер B
- Спикер C
- Спикер D

Темы:
1. Завершение встречи
   - Затронутые вопросы: благодарность за беседу, ценность встречи

2. Отсутствие давления со стороны правительства Эмиратов
   - Затронутые вопросы: отсутствие давления, 7 лет без давления

3. Необходимость занять позицию в современном мире
   - Затронутые вопросы: важность понимания других точек зрения, опасность разделения общества

4. Работа с агентством национальной безопасности
   - Затронутые вопросы: активность агентства, опыт работы с ним, прямой контакт

5. Использование каналов связи
   - Затронутые вопросы: популярность каналов, использование их президентом, легкость использования

Ключевые решения:
- Ключевые решения не принимались.

Важные даты:
- Мероприятия не планировались.


#### $\boxed{\text{Экспорт в файл}}$

In [61]:
with open('data/reports/report.txt', 'w', encoding='utf-8') as f:
    f.write(result)